In [8]:
# Libarary Installations

# !pip install pypyodbc
# !pip install numpy
# !pip install pandas


  Using cached pypyodbc-1.3.6.tar.gz (24 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pypyodbc: filename=pypyodbc-1.3.6-py3-none-any.whl size=22857 sha256=5af33fd9518cf38a98ddddb681e7d8c2184de29619b3bd591c5e9baeeb28d479
  Stored in directory: c:\users\hamil\appdata\local\pip\cache\wheels\4a\b0\00\a014827d8966a4b2601f6bff537634a1fe3e7b4269fe668879
Successfully built pypyodbc


In [14]:
# Imports

import pypyodbc as odbc # pip install pypyodbc
import numpy as np # pip install numpy
import pandas as pd # pip install pandas

In [11]:
# Connect to database

DRIVER_NAME =   "SQL SERVER"
SERVER_NAME = "LAPTOP-KBAD6AQA\SQLEXPRESS"
DATABASE_NAME = 'Netflix'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={{{SERVER_NAME}}};
    DATABASE={{{DATABASE_NAME}}};
    Trust_Connection=yes;
"""

conn = odbc.connect(connection_string)
print(conn)


In [16]:
try:
    # Establish a connection
    conn = odbc.connect(connection_string)

    # Create a cursor
    cursor = conn.cursor()

    # Example query
    query = "SELECT * FROM Streaming_Data"
    cursor.execute(query)

    # Fetch and print results
    rows = cursor.fetchall()

    # Convert tuples to numpy arrays
    arrays = [np.array(row) for row in rows]

    # Print the length of each row
    # for i, row in enumerate(arrays):
    #     print(f"Row {i + 1}: {len(row)} elements")

    # Create a DataFrame from the arrays
    columns = ["User_ID", "Session_ID", "Device_ID", "Video_ID", "Duration_Watched_minutes","Genre", "Country", "Age", "Gender", "Subscription_Status", "Ratings", "Languages", "Device_Type", "Location", "Playback_Quality", "Interaction_Events"]
    df = pd.DataFrame(arrays, columns=columns)

    # Print the DataFrame
    print(df.head())

except odbc.Error as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if conn:
        conn.close()

                                User_ID                            Session_ID  \
0  eb4f9229-74df-45f6-baac-cf19241b8b30  cb2142a7-0750-49ed-b8ac-a975fe1ff69a   
1  661d4b59-4328-410a-901c-1e3b4c40c334  3bc0a662-b353-4015-8b0c-55ceb510d13a   
2  dd3fe9e9-ea82-4891-ab93-8a47c80e3251  bd545b4a-9f54-4e87-b9f8-15ae20b44f22   
3  a1b3365b-1d00-4ddf-bc43-02fc9c10c680  0441086d-c59e-478d-a496-5c5b995ecfdb   
4  338d3f91-5f1c-4590-8803-324901826406  0295f01d-7f15-4799-856c-90c688697ef8   

  Device_ID Video_ID        Genre      Country  \
0       232       11  90.04452533       Sci-Fi   
1       549       85  68.97347929       Comedy   
2       844       50  42.51134326       Comedy   
3       201       38  53.31666005  Documentary   
4       700       31  69.43778589       Action   

                                Age Gender Subscription_Status  Ratings  \
0                             Sudan     56              Female  Premium   
1                              Cuba     52                Male